# BAF642 금융시계열 과제 1

20249433 최재필

## 1. 추세/계절성/잡음 분해
VIX와 V2X의 시계열을 추세, 계절성 및 잡음으로 분해하라.



## 2. 단위근 테스트 및 정상성 확인
VIX와 V2X 각각에 대해 단위근 테스트를 실행하고 정상성을 살펴보라.



## 3. ARIMA GARCH 모델 적용
VIX와 V2X 각각에 대해 ARIMA GARCH 모델을 적용하라.



## 4. Multivariate GARCH 모델 적용
VIX와 V2X에 대해 Multivariate GARCH 모델을 적용해보라.



## 5. VAR 모델 및 분산 분석
VIX와 V2X를 이용해 VAR 모델을 만들고, Granger causality 테스트를 실행하고, Impulse response 그래프와 분산 분해(Variance Decomposition)를 계산하고 그래프로 보여라.



## 6. VECM 모델 구축
VIX와 V2X를 이용해 공적분관계를 확인하고, VECM 모델을 구축하라.



## 7. 페어 트레이딩
트레이딩 경계(평균 +/- some 표준편차)를 설정하고 다음 방법으로 페어 트레이딩을 수행하라.



### 1) 단순 스프레드를 이용한 페어 트레이딩



### 2) 공적분관계를 이용한 페어 트레이딩



### 3) 칼만 필터를 이용한 페어 트레이딩



## 8. 최소 총 이익(MTP) 경계 및 백테스트
최소 총 이익(MTP) 경계를 구하고 이를 이용한 백테스트 결과를 제시하라. (논문과 블로그 참조)
